This notebook is used to get geo locations for addresses in the housing dataset.
WARNING: Does not currently remove duplicate properties

Reads `Ames_Real_Estate_Data.csv` and `Ames_Housing_Price_Data.csv`
Writes `housing_geolocation.csv` with location columns `latitude` and `longitude`

In [1]:
# See: https://geopy.readthedocs.io/en/stable/#usage-with-pandas

import pandas as pd
from geopy import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm

# Show progress bar
tqdm.pandas()

In [2]:
# TODO check: Do we need anything besides Prop_Addr from the real estate data?
df = pd.read_csv('../data/Ames_Real_Estate_Data.csv', usecols=['MapRefNo', 'Prop_Addr'])
housing = pd.read_csv('../data/Ames_Housing_Price_Data.csv', index_col=0)
merged = pd.merge(left=housing, right=df,
                  left_on="PID", right_on="MapRefNo")
# NOTE: We have not removed dupes above TODO fix


In [3]:
print(f"Housing data has {len(set(housing.PID))} unique PIDs, but {len(housing.PID)} total PIDs")
print(f"Address data has {len(set(df.MapRefNo))} unique MapRefNos, but {len(df.MapRefNo)} total MapRefNos\n")

print(f"<---- WARNING WARNING WARNING ----> \nMerge found {merged.shape[0]} properties with addresses out of {housing.shape[0]} total")

Housing data has 2579 unique PIDs, but 2580 total PIDs
Address data has 14235 unique MapRefNos, but 22213 total MapRefNos

<---- WARNING WARNING WARNING ----> 
Merge found 2603 properties with addresses out of 2580 total


In [4]:
# Use Nominatim as geolocator provider
geolocator = Nominatim(user_agent="ames-iowa-nycdsa-1")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Get location from provider
merged['Prop_Addr_Full'] = merged['Prop_Addr'] + ", Ames, USA"
merged['geolocation'] = merged['Prop_Addr_Full'].progress_apply(geocode)

In [17]:
# Extract lat/lon from location
merged['latitude'] = merged['geolocation'].apply(lambda loc: loc.latitude if loc else None)
merged['longitude'] = merged['geolocation'].apply(lambda loc: loc.longitude if loc else None)

In [25]:
merged.to_csv("../data/housing_geolocation.csv")